# Final Project 2

## San Francisco Crime Classification

### Project Problem and Hypothesis

##### The following is an excerpt from Kaggle San Francisco Crime Classification: 

##### With rising wealth inequality, housing shortages, and a proliferation of expensive digital toys riding BART to work, there is no scarcity of crime in the city by the bay

##### My overall summary:
By using this dataset, predict the category of crimes that occurred in the city. To keep this project manageable, I've reduced to the number of categories I'm allowed to predict: predict whether the crime will fall under "LARCENY/THEFT" or not. One example of how this prediction  could help in the future is to allocate enough law enforcement at the right place and right time to prevent theft. Through some exploratory data analysis (and looking at the test set), I'll keep the columns,"ID", "Dates", "Category", "PdDistrict", "Address","X", and "Y". Overall with the limited amount of columns to work with, with some data minpulation, I think PDdistrict, Dates, X, and Y will have the biggest impact in predictive power.

### Datasets

Dates - timestamp of the crime incident. I will try to break into more columns for Day, Year, Month, and time.

Category - category of the crime incident (only in train.csv). This is the target variable I will try to predict.

Descript - detailed description of the crime incident (only in train.csv). I am dropping this column.

DayOfWeek - the day of the week, Sunday through Saturday. I created dummy variables for this.

PdDistrict - name of the Police Department District

Resolution - how the crime incident was resolved (only in train.csv). I might drop this column.

Address - the approximate street address of the crime incident

X - Longitude

Y - Latitude

I'll also create dummy variables for DayOfWeek and Address.

### Domain knowledge

Thankfully, if I google "San Francisco Crime Classification github," there is a lot of work that has already been done on this, so I was leverage other ideas if I get stuck or need help creating a model.
    
<b>Benchmark</b>: 

After doing some google searches, it looks like the best model for this classifier is from a decision tree, which we haven't covered yet in class. I read someone online that someone got a rank of 1149 with a score of 2.64. I'm not sure what this means, I'm I am adding this comment as part of the document writeup.

### Project Concerns

I'm not sure where to start. I plan on working with the same model my group had used for the class assignment, titantic, and work from there to see what can be done. I'm also reading from other folks who used predicitive models that we haven't covered in class (yet at least). Some of these include Naïve Bayes and decision trees. While there are other models out there that were used for this dataset, I'd like to stick with materials we learned in class as much as possible. I need help determining what would be the best route. 

I don't what to do with X and Y variables, although I think they're important to keep.

<b>Assumptions</b>: 

The only high level assumption I can think of for this is that the data entered is as accurate as possible. 

<b>Data I wish I had</b>:

Not to get carried away, but I read from another person had data for nearby police stations. I think this is possible to get, but would take a lot of time to do as I would take ages to figure out how to pull this information together.

Also, if I could somehow understand how coordinates work, I could create a range of coordinates and add it into my data to add more classifications.

<b>Risks</b>:

At the moment, the only risk I can think of is that the data entry is already biased. At the same time, I think this risk is low, because I don't need the folks entering data have an agenda to skew it, unless the data released by SFPD was manipulated.



### Outcomes

I'm hoping to train the training set and when compared back to the test set, my goal to get a good percentage of them predicted correctly. Given that I'm obviously new to data science, I will try to keep my model as simple as possible using material from class. My definition of success would be to come up with predictions utilizing class material. My best case sceanrio for success is a high score for predictions.



In [40]:
import os

import math

import numpy as np
import pandas as pd
from datetime import datetime as dt
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 12)

from sklearn import linear_model, cross_validation, tree, ensemble, metrics, model_selection, externals
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

#take out x and y because it's probably not useful
#probably use logisitic regression.
#will go into decision trees so this might work.
#You can plot x and y.

In [41]:
df = pd.read_csv("train.csv")

In [42]:
df

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541
...,...,...,...,...,...,...,...,...,...
878044,2003-01-06 00:15:00,ROBBERY,ROBBERY ON THE STREET WITH A GUN,Monday,TARAVAL,NONE,FARALLONES ST / CAPITOL AV,-122.459033,37.714056
878045,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,INGLESIDE,NONE,600 Block of EDNA ST,-122.447364,37.731948
878046,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,SOUTHERN,NONE,5TH ST / FOLSOM ST,-122.403390,37.780266
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607


### Exploratory analysis

In [43]:
df.Address.value_counts() #count the number of times an address occurs.
# Most crime occurs on Bryant ST

800 Block of BRYANT ST      26533
800 Block of MARKET ST       6581
2000 Block of MISSION ST     5097
1000 Block of POTRERO AV     4063
900 Block of MARKET ST       3251
                            ...  
2600 Block of TARAVAL ST        1
0 Block of PALOALTO AV          1
ALEMANY BL / RAMSELL ST         1
2800 Block of MARKET ST         1
KRAMER PL / GREENWICH ST        1
Name: Address, dtype: int64

In [44]:
df.columns # see all features

Index([u'Dates', u'Category', u'Descript', u'DayOfWeek', u'PdDistrict',
       u'Resolution', u'Address', u'X', u'Y'],
      dtype='object')

In [45]:
df.Resolution.unique() # see all unique resolutions 

array(['ARREST, BOOKED', 'NONE', 'ARREST, CITED', 'PSYCHOPATHIC CASE',
       'JUVENILE BOOKED', 'UNFOUNDED', 'EXCEPTIONAL CLEARANCE', 'LOCATED',
       'CLEARED-CONTACT JUVENILE FOR MORE INFO', 'NOT PROSECUTED',
       'JUVENILE DIVERTED', 'COMPLAINANT REFUSES TO PROSECUTE',
       'JUVENILE ADMONISHED', 'JUVENILE CITED',
       'DISTRICT ATTORNEY REFUSES TO PROSECUTE',
       'PROSECUTED BY OUTSIDE AGENCY', 'PROSECUTED FOR LESSER OFFENSE'], dtype=object)

In [46]:
df.Category.unique() # see all unique categories of crimes 

array(['WARRANTS', 'OTHER OFFENSES', 'LARCENY/THEFT', 'VEHICLE THEFT',
       'VANDALISM', 'NON-CRIMINAL', 'ROBBERY', 'ASSAULT', 'WEAPON LAWS',
       'BURGLARY', 'SUSPICIOUS OCC', 'DRUNKENNESS',
       'FORGERY/COUNTERFEITING', 'DRUG/NARCOTIC', 'STOLEN PROPERTY',
       'SECONDARY CODES', 'TRESPASS', 'MISSING PERSON', 'FRAUD',
       'KIDNAPPING', 'RUNAWAY', 'DRIVING UNDER THE INFLUENCE',
       'SEX OFFENSES FORCIBLE', 'PROSTITUTION', 'DISORDERLY CONDUCT',
       'ARSON', 'FAMILY OFFENSES', 'LIQUOR LAWS', 'BRIBERY',
       'EMBEZZLEMENT', 'SUICIDE', 'LOITERING', 'SEX OFFENSES NON FORCIBLE',
       'EXTORTION', 'GAMBLING', 'BAD CHECKS', 'TREA', 'RECOVERED VEHICLE',
       'PORNOGRAPHY/OBSCENE MAT'], dtype=object)

In [47]:
df.Category.value_counts() # see 5 most popular crimes

LARCENY/THEFT                174900
OTHER OFFENSES               126182
NON-CRIMINAL                  92304
ASSAULT                       76876
DRUG/NARCOTIC                 53971
                              ...  
EXTORTION                       256
SEX OFFENSES NON FORCIBLE       148
GAMBLING                        146
PORNOGRAPHY/OBSCENE MAT          22
TREA                              6
Name: Category, dtype: int64

In [48]:
df = df.drop(['Descript'], axis = 1) # drop Descript column

In [49]:
df.head() # check columns

,Dates,Category,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [50]:
df.PdDistrict.unique().tolist() # show all unique districts

['NORTHERN',
 'PARK',
 'INGLESIDE',
 'BAYVIEW',
 'RICHMOND',
 'CENTRAL',
 'TARAVAL',
 'TENDERLOIN',
 'MISSION',
 'SOUTHERN']

In [51]:
#No longer created dummy variables. Left code below as point of reference.
#PdDistrict_df = pd.get_dummies(df.PdDistrict, prefix = 'District') 

In [52]:
#No longer creating dummy variable columns. Left code below as point of reference.
#df = df.join(PdDistrict_df)

In [53]:
df.head()

,Dates,Category,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [15]:
#No longer created dummy variables. Left code below as point of reference.
#DayOfWeek_df = pd.get_dummies(df.DayOfWeek, prefix = 'DayOfWeek')

In [16]:
#No longer created dummy variables. Left code below as point of reference.
#df = df.join(DayOfWeek_df)

#df.head()

In [54]:
#testing out daytime package. Run the time right now.
now = dt.now().strftime("%Y-%m-%d_%H:%M:%S.%f")

print dt.now()

now

2017-06-23 15:49:59.460295


'2017-06-23_15:49:59.460224'

In [58]:
df['Dates'] = pd.to_datetime(df['Dates']) # Convert original Date column to correct type to use striftime below

In [59]:
#df['Dates2'] = df['Dates'].map(lambda x: x.strftime('%Y-%m-%d')) this code works
df['Year'] = df['Dates'].map(lambda x: x.strftime('%Y'))

In [60]:
df['Year']

0         2015
1         2015
2         2015
3         2015
4         2015
          ... 
878044    2003
878045    2003
878046    2003
878047    2003
878048    2003
Name: Year, dtype: object

In [61]:
df.head()

,Dates,Category,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Year
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,2015
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,2015


In [62]:
df['Month'] = df['Dates'].map(lambda x: x.strftime('%m'))
df['Day'] = df['Dates'].map(lambda x: x.strftime('%d'))
df['Hour'] = df['Dates'].map(lambda x: x.strftime('%H'))
df['Minute'] = df['Dates'].map(lambda x: x.strftime('%M'))


In [64]:
df.head()

,Dates,Category,DayOfWeek,PdDistrict,Resolution,Address,...,Y,Year,Month,Day,Hour,Minute
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,...,37.774599,2015,05,13,23,53
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,...,37.774599,2015,05,13,23,53
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,...,37.800414,2015,05,13,23,33
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,...,37.800873,2015,05,13,23,30
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,NONE,100 Block of BRODERICK ST,...,37.771541,2015,05,13,23,30


In [65]:
df.Category.unique().tolist() # show all unique Categories

['WARRANTS',
 'OTHER OFFENSES',
 'LARCENY/THEFT',
 'VEHICLE THEFT',
 'VANDALISM',
 'NON-CRIMINAL',
 'ROBBERY',
 'ASSAULT',
 'WEAPON LAWS',
 'BURGLARY',
 'SUSPICIOUS OCC',
 'DRUNKENNESS',
 'FORGERY/COUNTERFEITING',
 'DRUG/NARCOTIC',
 'STOLEN PROPERTY',
 'SECONDARY CODES',
 'TRESPASS',
 'MISSING PERSON',
 'FRAUD',
 'KIDNAPPING',
 'RUNAWAY',
 'DRIVING UNDER THE INFLUENCE',
 'SEX OFFENSES FORCIBLE',
 'PROSTITUTION',
 'DISORDERLY CONDUCT',
 'ARSON',
 'FAMILY OFFENSES',
 'LIQUOR LAWS',
 'BRIBERY',
 'EMBEZZLEMENT',
 'SUICIDE',
 'LOITERING',
 'SEX OFFENSES NON FORCIBLE',
 'EXTORTION',
 'GAMBLING',
 'BAD CHECKS',
 'TREA',
 'RECOVERED VEHICLE',
 'PORNOGRAPHY/OBSCENE MAT']

In [66]:
#recategorize boarder categories

#df['HighLevelCategory'] = df.Category.apply(lambda NameOfCategory:"Theft" if NameOfCategory =="LARCENY/THEFT" else "Other")

#df['HighLevelCategory'] = df.Category.apply(lambda NameOfCategory:"Theft" 
                                            #if NameOfCategory =="LARCENY/THEFT" or NameOfCategory == "VEHICLE THEFT" or NameOfCategory == "BURGLARY" or NameOfCategory == "STOLEN PROPERTY" or NameOfCategory == "Fraud" 
                                            #else "Other")

#df['HighLevelCategory'] = df.Category.apply(lambda NameOfCategory:"Theft" 
                                           # if NameOfCategory =="LARCENY/THEFT" or NameOfCategory == "VEHICLE THEFT" or NameOfCategory == "BURGLARY" or NameOfCategory == "STOLEN PROPERTY" or NameOfCategory == "Fraud" 
                                            #else ("ASSAULT" if NameOfCategory == "ASSAULT" else ("NARCOTIC" if NameOfCategory == "NARCOTIC" else "Other")))
        
def category_to_high_level(Category):
    if Category in ["LARCENY/THEFT", "VEHICLE THEFT", "BURGLARY", "STOLEN PROPERTY", "FRAUD"]:
       return "Theft"
    elif Category in ["ASSAULT"]:
       return "Assault"
    elif Category in ["DRUG/NARCOTIC"]:
       return "Drug/Narcotic"               
    else:
       return "Other"
                    
df['HighLevelCategory'] = df.Category.apply(category_to_high_level)

def DayOfWeek_to_numbers(week):
    if week in ["Sunday"]:
       return "1"
    elif week in ["Monday"]:
       return "2"
    elif week in ["Tuesday"]:
       return "3"
    elif week in ["Wednesday"]:
       return "4"
    elif week in ["Thursday"]:
       return "5"
    elif week in ["Friday"]:
       return "6"             
    else:
       return "7"
                    
df['DayInNumber'] = df.DayOfWeek.apply(DayOfWeek_to_numbers)



In [67]:
df.HighLevelCategory.unique().tolist() #check

['Other', 'Theft', 'Assault', 'Drug/Narcotic']

In [68]:
df.DayInNumber.unique().tolist() #check

['4', '3', '2', '1', '7', '6', '5']

In [69]:
#df = df.drop(['HighLevelCategory'], axis = 1)

In [70]:
df.head()

,Dates,Category,DayOfWeek,PdDistrict,Resolution,Address,...,Month,Day,Hour,Minute,HighLevelCategory,DayInNumber
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,...,05,13,23,53,Other,4
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,...,05,13,23,53,Other,4
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,...,05,13,23,33,Other,4
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,...,05,13,23,30,Theft,4
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,NONE,100 Block of BRODERICK ST,...,05,13,23,30,Theft,4


In [71]:
df.HighLevelCategory.value_counts()

Other            460547
Theft            286655
Assault           76876
Drug/Narcotic     53971
Name: HighLevelCategory, dtype: int64

In [72]:
X = df[df.columns.values]
X.drop(['HighLevelCategory','Address','Category','DayOfWeek','PdDistrict','Resolution','Dates'], axis = 1, inplace = True) #dropped address because of model error RandomForestClassifier

y = df.HighLevelCategory

X

,X,Y,Year,Month,Day,Hour,Minute,DayInNumber
0,-122.425892,37.774599,2015,05,13,23,53,4
1,-122.425892,37.774599,2015,05,13,23,53,4
2,-122.424363,37.800414,2015,05,13,23,33,4
3,-122.426995,37.800873,2015,05,13,23,30,4
4,-122.438738,37.771541,2015,05,13,23,30,4
...,...,...,...,...,...,...,...,...
878044,-122.459033,37.714056,2003,01,06,00,15,2
878045,-122.447364,37.731948,2003,01,06,00,01,2
878046,-122.403390,37.780266,2003,01,06,00,01,2
878047,-122.390531,37.780607,2003,01,06,00,01,2


In [73]:
y

0         Other
1         Other
2         Other
3         Theft
4         Theft
          ...  
878044    Other
878045    Theft
878046    Theft
878047    Other
878048    Other
Name: HighLevelCategory, dtype: object

### Split dataset into a training and testing sets (60%, 40%)

In [74]:
train_X, test_X, train_y, test_y = model_selection.train_test_split(X, y, train_size = .6, random_state = 0)


In [75]:
model = ensemble.RandomForestClassifier()

params = {"n_estimators": [1, 10, 100],
          "min_samples_leaf": [1, 5, 10]}

gs = GridSearchCV(model, params, cv = 3)

gs.fit (train_X, train_y)

#code above figures out best combinations


GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [1, 10, 100], 'min_samples_leaf': [1, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [76]:
gs.best_params_ # this displays the best combinations

{'min_samples_leaf': 5, 'n_estimators': 100}

In [77]:
best_forest = gs.best_estimator_ #this IS the best model. It's shorter version of Ivan's example.

best_forest.fit(train_X, train_y) # reports the default parameters.

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [78]:
train_y_hat1 = model_selection.cross_val_predict(best_forest, train_X, train_y, cv = 5) #remember that cross validation already does a .fit


In [79]:
pd.crosstab(train_y_hat1, train_y, rownames = ['Hypothesized Class'], colnames = ['True Class'])

True Class,Assault,Drug/Narcotic,Other,Theft
Hypothesized Class,,,,
Assault,143,4,235,11
Drug/Narcotic,167,3282,3017,133
Other,38648,27865,222271,82075
Theft,7039,1232,50465,90242


In [80]:
train_y_hat2 = best_forest.predict(train_X)

In [81]:
pd.crosstab(train_y_hat2, train_y, rownames = ['Hypothesized Class'], colnames = ['True Class'])

True Class,Assault,Drug/Narcotic,Other,Theft
Hypothesized Class,,,,
Assault,1849,9,62,21
Drug/Narcotic,152,9585,1039,146
Other,37462,21616,259227,37186
Theft,6534,1173,15660,135108


In [82]:
best_forest.oob_score_
#how well are you doing on training data you're not using on your bootstrap.

AttributeError: 'RandomForestClassifier' object has no attribute 'oob_score_'

### 5-fold cross-validation error on your training set

In [87]:
model_selection.cross_val_score(best_forest, train_X, train_y, cv = 5).mean()

0.59967656602469288

### Training Scores

In [88]:
metrics.accuracy_score(train_y, train_y_hat1)

0.59969743503110118

In [89]:
metrics.accuracy_score(train_y, train_y_hat2)

0.77021006816253468

In [90]:
best_forest.score(train_X, train_y)

0.77021006816253468

### Test 

In [91]:
best_forest.score(test_X, test_y)

0.60219520528443715

In [92]:
test_y_hat1 = model_selection.cross_val_predict(best_forest, test_X, test_y, cv = 5) #remember that cross validation already does a .fit


In [93]:
pd.crosstab(test_y_hat1, test_y, rownames = ['Hypothesized Class'], colnames = ['True Class'])

True Class,Assault,Drug/Narcotic,Other,Theft
Hypothesized Class,,,,
Assault,40,1,87,4
Drug/Narcotic,90,1805,1634,76
Other,26051,18884,149688,57087
Theft,4698,898,33150,57027


In [94]:
test_y_hat2 = best_forest.predict(test_X)

In [95]:
pd.crosstab(test_y_hat2, test_y, rownames = ['Hypothesized Class'], colnames = ['True Class'])

True Class,Assault,Drug/Narcotic,Other,Theft
Hypothesized Class,,,,
Assault,124,9,236,15
Drug/Narcotic,96,2519,2312,83
Other,26142,18244,148302,53538
Theft,4517,816,33709,60558


In [96]:
metrics.accuracy_score(test_y, test_y_hat1)

0.59381584192244175

In [97]:
metrics.accuracy_score(test_y, test_y_hat2)

0.60219520528443715

In [84]:
#df.columns

Index([u'Dates', u'Category', u'DayOfWeek', u'PdDistrict', u'Resolution',
       u'Address', u'X', u'Y', u'District_BAYVIEW', u'District_CENTRAL',
       u'District_INGLESIDE', u'District_MISSION', u'District_NORTHERN',
       u'District_PARK', u'District_RICHMOND', u'District_SOUTHERN',
       u'District_TARAVAL', u'District_TENDERLOIN', u'DayOfWeek_Friday',
       u'DayOfWeek_Monday', u'DayOfWeek_Saturday', u'DayOfWeek_Sunday',
       u'DayOfWeek_Thursday', u'DayOfWeek_Tuesday', u'DayOfWeek_Wednesday'],
      dtype='object')

In [88]:
#df[['DayOfWeek_Saturday', 'DayOfWeek_Sunday', 'DayOfWeek_Monday' , 
    #'DayOfWeek_Tuesday' , 'DayOfWeek_Wednesday', 'DayOfWeek_Thursday', 
    #'DayOfWeek_Friday']].sum()

DayOfWeek_Saturday     126810
DayOfWeek_Sunday       116707
DayOfWeek_Monday       121584
DayOfWeek_Tuesday      124965
DayOfWeek_Wednesday    129211
DayOfWeek_Thursday     125038
DayOfWeek_Friday       133734
dtype: int64

In [60]:
#df['LARCENY/THEFT'] = df.Category.apply(lambda x:1 if x =="LARCENY/THEFT" else 0)

In [90]:
#df.head()

,Dates,Category,DayOfWeek,PdDistrict,Resolution,Address,...,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday,LARCENY/THEFT
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,...,0,0,0,0,1,0
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,...,0,0,0,0,1,0
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,...,0,0,0,0,1,0
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,...,0,0,0,0,1,1
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,NONE,100 Block of BRODERICK ST,...,0,0,0,0,1,1


In [91]:
#df.corr()

,X,Y,District_BAYVIEW,District_CENTRAL,District_INGLESIDE,District_MISSION,...,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday,LARCENY/THEFT
X,1.000000,0.559338,0.324589,0.141814,-0.063299,0.042235,...,0.005575,0.002131,-0.001416,-0.002643,0.000631,0.002808
Y,0.559338,1.000000,-0.021009,0.019922,-0.028753,-0.009247,...,-0.000597,-0.000958,0.000523,0.000236,0.000109,0.007560
District_BAYVIEW,0.324589,-0.021009,1.000000,-0.110578,-0.105772,-0.133924,...,-0.005097,0.000867,-0.003549,0.000949,0.003333,-0.072548
District_CENTRAL,0.141814,0.019922,-0.110578,1.000000,-0.103137,-0.130589,...,0.018852,0.009484,-0.005903,-0.009609,-0.008992,0.077320
District_INGLESIDE,-0.063299,-0.028753,-0.105772,-0.103137,1.000000,-0.124913,...,-0.006142,-0.001570,0.000799,0.003702,0.000499,-0.054552
...,...,...,...,...,...,...,...,...,...,...,...,...,...
DayOfWeek_Sunday,0.002131,-0.000958,0.000867,0.009484,-0.001570,-0.000623,...,-0.160859,1.000000,-0.159543,-0.159489,-0.162635,0.007584
DayOfWeek_Thursday,-0.001416,0.000523,-0.003549,-0.005903,0.000799,-0.000289,...,-0.167420,-0.159543,1.000000,-0.165994,-0.169269,-0.004011
DayOfWeek_Tuesday,-0.002643,0.000236,0.000949,-0.009609,0.003702,0.002056,...,-0.167363,-0.159489,-0.165994,1.000000,-0.169211,-0.007631
DayOfWeek_Wednesday,0.000631,0.000109,0.003333,-0.008992,0.000499,0.000671,...,-0.170665,-0.162635,-0.169269,-0.169211,1.000000,-0.010068


In [92]:
#df.drop(['X', 'Y'], axis = 1).corr()

,District_BAYVIEW,District_CENTRAL,District_INGLESIDE,District_MISSION,District_NORTHERN,District_PARK,...,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday,LARCENY/THEFT
District_BAYVIEW,1.000000,-0.110578,-0.105772,-0.133924,-0.124307,-0.082145,...,-0.005097,0.000867,-0.003549,0.000949,0.003333,-0.072548
District_CENTRAL,-0.110578,1.000000,-0.103137,-0.130589,-0.121211,-0.080100,...,0.018852,0.009484,-0.005903,-0.009609,-0.008992,0.077320
District_INGLESIDE,-0.105772,-0.103137,1.000000,-0.124913,-0.115943,-0.076618,...,-0.006142,-0.001570,0.000799,0.003702,0.000499,-0.054552
District_MISSION,-0.133924,-0.130589,-0.124913,1.000000,-0.146803,-0.097011,...,-0.000985,-0.000623,-0.000289,0.002056,0.000671,-0.047016
District_NORTHERN,-0.124307,-0.121211,-0.115943,-0.146803,1.000000,-0.090045,...,0.002791,0.002844,0.000877,-0.004193,-0.002356,0.067200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
DayOfWeek_Sunday,0.000867,0.009484,-0.001570,-0.000623,0.002844,0.001333,...,-0.160859,1.000000,-0.159543,-0.159489,-0.162635,0.007584
DayOfWeek_Thursday,-0.003549,-0.005903,0.000799,-0.000289,0.000877,0.000348,...,-0.167420,-0.159543,1.000000,-0.165994,-0.169269,-0.004011
DayOfWeek_Tuesday,0.000949,-0.009609,0.003702,0.002056,-0.004193,-0.002085,...,-0.167363,-0.159489,-0.165994,1.000000,-0.169211,-0.007631
DayOfWeek_Wednesday,0.003333,-0.008992,0.000499,0.000671,-0.002356,0.000339,...,-0.170665,-0.162635,-0.169269,-0.169211,1.000000,-0.010068
